In [1]:
# --- preprocess_data.py ---
import numpy as np
import pandas as pd
import math
import sklearn
import sklearn.preprocessing
import datetime
from scipy.stats.mstats import winsorize

pd.options.mode.chained_assignment = None

# *** DEFINE HELPER FUNCTION HERE ***
def find_col(df, potential_names, default=None):
    """Helper to find the first matching column name from a list."""
    for name in potential_names:
        if name in df.columns: return name
    print(f"  Warning: Could not find column using names: {potential_names}") # Added warning
    return default
# *** END HELPER FUNCTION DEFINITION ***

# --- Configuration ---
INPUT_FILE = "Cleaned_OSEFX_Market_Macro_Data.csv"
OUTPUT_FILE = "Cleaned_OSEFX_Market_Macro_Data_PREPROCESSED.csv" # Output for the pipeline
TARGET_COL_NAME = "TargetReturn_t+1"         # Standard name for the pipeline
NEXT_RAW_RET_COL_NAME = "NextMonthlyReturn_t+1" # Raw return for portfolio eval
MKT_CAP_ORIG_COL_NAME = "MarketCap_orig"        # Original market cap for portfolio eval

# --- Load Data ---
print(f"Loading raw data from: {INPUT_FILE}")
try:
    df = pd.read_csv(INPUT_FILE)
    df["Date"] = pd.to_datetime(df["Date"])
    print(f"Raw data loaded. Shape: {df.shape}")
except FileNotFoundError:
    print(f"ERROR: File not found: {INPUT_FILE}"); exit()
except Exception as e:
    print(f"ERROR loading data: {e}"); exit()

# --- Data Preparation ---
print("Sorting and preparing columns...")
df = df.sort_values(by=["Instrument", "Date"]).reset_index(drop=True)

# *** COLUMN CLEANING STEP ***
print("  Cleaning original column names...")
df.columns = df.columns.str.replace("[^A-Za-z0-9_]+", "_", regex=True).str.strip('_').str.replace('__', '_')
# Find the potentially renamed columns AFTER cleaning
norges_bank_10y_col = find_col(df, ['NorgesBank10Y', 'norgesbank10y']) # Helper function is now defined
if not norges_bank_10y_col:
    print("ERROR: Cannot find NorgesBank10Y column after cleaning.")
    exit()
close_price_col = find_col(df, ['ClosePrice', 'closeprice'])
common_shares_col = find_col(df, ['CommonSharesOutstanding', 'commonsharesoutstanding'])
if not close_price_col or not common_shares_col:
     print(f"ERROR: Cannot find ClosePrice/CommonSharesOutstanding after cleaning.")
     exit()
nibor3m_col = find_col(df, ['NIBOR3M', 'nibor3m'])
if not nibor3m_col:
    print("ERROR: Cannot find NIBOR3M column after cleaning.")
    exit()
print("  Original column names cleaned.")
# *** END OF CLEANING AND FINDING RENAMED COLS ***

# --- Continue with Calculations using found column names ---
# Calculate Monthly Return (t) and Winsorize EARLY
print("Calculating returns...")
df["MonthlyReturn_t"] = df.groupby("Instrument")[close_price_col].pct_change() # Use found name
# ... (rest of return calculation as before) ...
df["MonthlyReturn_t"].replace([np.inf, -np.inf], np.nan, inplace=True)
df["MonthlyReturn_t"].fillna(0, inplace=True)
df["MonthlyReturn_t"] = winsorize(df["MonthlyReturn_t"].values, limits=[0.01, 0.01])
print("  MonthlyReturn_t calculated and winsorized.")

# Calculate Risk-Free Rate (t) using the cleaned column name
df.loc[:, "MonthlyRiskFreeRate_t"] = df[norges_bank_10y_col] / 12 / 100 # Use found name
print("  MonthlyRiskFreeRate_t calculated.")

# Calculate Adjusted Return (Excess Return for month t)
df["AdjustedReturn_t"] = df["MonthlyReturn_t"] - df["MonthlyRiskFreeRate_t"]
print("  AdjustedReturn_t (Excess Return t) calculated.")

# --- Create Target and Necessary Lead Variables ---
# *** NOW THESE COLUMNS WILL BE CREATED WITH '+' ***
print("Calculating lead variables (Target and Next Raw Return)...")
df[TARGET_COL_NAME] = df.groupby("Instrument")["AdjustedReturn_t"].shift(-1)
print(f"  Target variable '{TARGET_COL_NAME}' created.")
df[NEXT_RAW_RET_COL_NAME] = df.groupby("Instrument")["MonthlyReturn_t"].shift(-1)
print(f"  Next raw return variable '{NEXT_RAW_RET_COL_NAME}' created.")
# *** END OF TARGET CREATION ***

# Drop rows where the TARGET variable is NaN (essential!)
initial_rows = len(df)
df.dropna(subset=[TARGET_COL_NAME], inplace=True)
print(f"  Dropped {initial_rows - len(df)} rows with missing target '{TARGET_COL_NAME}'.")
if df.empty: print("ERROR: DataFrame empty after dropping missing target."); exit()

# --- Feature Engineering ---
print("Performing feature engineering (Market Cap, Term Spread, Log Transforms)...")
# Recalculate MarketCap (t) using cleaned column names
df["MarketCap"] = df[close_price_col] * df[common_shares_col] # Use found names
df.loc[df["MarketCap"] <= 0, "MarketCap"] = np.nan
df[MKT_CAP_ORIG_COL_NAME] = df["MarketCap"].copy()
print(f"  MarketCap calculated and original stored in '{MKT_CAP_ORIG_COL_NAME}'.")

# Create term spread (t) using cleaned column names
df["TermSpread"] = df[norges_bank_10y_col] - df[nibor3m_col] # Use found names
print("  TermSpread calculated.")

# Log-transform relevant variables (using potentially cleaned names)
vars_to_log = ["MarketCap", "BM", "ClosePrice", "Volume", "CommonSharesOutstanding"]
print(f"  Log-transforming: {vars_to_log}")
for var in vars_to_log:
    # Find the potentially cleaned column name - use more robust check now
    potential_names = [var, var.lower(), var.replace("_","")] # Add different variations if needed
    cleaned_var_name = find_col(df, potential_names)
    if cleaned_var_name:
        df[cleaned_var_name] = pd.to_numeric(df[cleaned_var_name], errors='coerce')
        original_nan_mask = df[cleaned_var_name].isna()
        log_col = f"log_{cleaned_var_name}" # Create log name based on found name
        df[log_col] = np.nan
        positive_mask = (~original_nan_mask) & (df[cleaned_var_name] > 1e-9)
        df.loc[positive_mask, log_col] = np.log(df.loc[positive_mask, cleaned_var_name])
        print(f"    - Logged {positive_mask.sum()} positive values for {cleaned_var_name} -> {log_col}.")
    else:
        print(f"    - Warning: Column for '{var}' not found for log transform (using {potential_names}).")


# --- Final Checks and Save ---
print("Final checks and saving preprocessed data...")
# Ensure essential ID/Date columns have standard names if possible
if 'Instrument' not in df.columns and 'instrument' in df.columns:
    df = df.rename(columns={'instrument': 'Instrument'})
if 'Date' not in df.columns and 'date' in df.columns:
    df = df.rename(columns={'date': 'Date'})

# Verify essential columns exist before saving
essential_final = ['Date', 'Instrument', TARGET_COL_NAME, NEXT_RAW_RET_COL_NAME, MKT_CAP_ORIG_COL_NAME]
missing = [c for c in essential_final if c not in df.columns]
if missing:
    print(f"ERROR: Essential columns missing before saving: {missing}")
    print(f"Available columns: {df.columns.tolist()}")
    exit()

# Drop rows where original market cap is non-positive or NaN
initial_rows = len(df)
df = df.dropna(subset=[MKT_CAP_ORIG_COL_NAME])
df = df[df[MKT_CAP_ORIG_COL_NAME] > 0]
rows_removed = initial_rows - len(df)
if rows_removed > 0:
    print(f"  Dropped {rows_removed} rows with missing or non-positive '{MKT_CAP_ORIG_COL_NAME}'.")

if df.empty: print("ERROR: DataFrame empty after final checks."); exit()

# Convert numeric columns to float32
numeric_cols_final = df.select_dtypes(include=[np.number]).columns
df[numeric_cols_final] = df[numeric_cols_final].astype("float32")

df.to_csv(OUTPUT_FILE, index=False)
print(f"Preprocessing complete. Final shape: {df.shape}")
print(f"Preprocessed data saved to: {OUTPUT_FILE}")
print("\nFinal Data Info:")
df.info(verbose=True)

Loading raw data from: Cleaned_OSEFX_Market_Macro_Data.csv
ERROR: File not found: Cleaned_OSEFX_Market_Macro_Data.csv
Sorting and preparing columns...


NameError: name 'df' is not defined